In [82]:
import matplotlib.pyplot as plt
import sklearn.metrics
import properscoring as ps

In [10]:
import xarray as xr
weather = xr.open_mfdataset("./combiprecip_201805.nc", combine='by_coords')

In [11]:
import xarray as xr
import pyproj

cosmo = xr.open_zarr("./cosmoe_prec_201805.zarr")

# Transform to the other coordinate system
src_proj = pyproj.Proj("EPSG:4326") # WSG84
dst_proj = pyproj.Proj("EPSG:21781") # CH1903 / LV03 
src_x = cosmo.lon.values
src_y = cosmo.lat.values
dst_x, dst_y = pyproj.transform(src_proj, dst_proj, src_x, src_y, always_xy=True)
cosmo = cosmo.assign_coords({"chx": (("y", "x"), dst_x) , "chy": (("y", "x"), dst_y)})


/home/goovi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # This is added back by InteractiveShellApp.init_path()


In [12]:
# Actual weather: (710, 640)
# Let's set this to 2018-05-01T01:00:00 (time=0)
weather.isel(time=0, dummy=0)
#weather?

<xarray.Dataset>
Dimensions:            (chx: 710, chy: 640)
Coordinates:
    dummy              float64 1.0
  * chx                (chx) float64 2.555e+05 2.565e+05 ... 9.635e+05 9.645e+05
  * chy                (chy) float64 -1.595e+05 -1.585e+05 ... 4.795e+05
    time               datetime64[ns] 2018-05-01T01:00:00
    lon                (chy, chx) float32 dask.array<chunksize=(640, 710), meta=np.ndarray>
    lat                (chy, chx) float32 dask.array<chunksize=(640, 710), meta=np.ndarray>
Data variables:
    swiss_coordinates  float64 dask.array<chunksize=(), meta=np.ndarray>
    RR                 (chy, chx) float32 dask.array<chunksize=(640, 710), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.6
    institution:               Federal Office of Meteorology and Climatology ...
    References:                
    history:                   Wed Sep 16 07:54:48 2020: ncrcat -O combipreci...
    NCO:                       "4.5.4"
    nco_openmp_thread_number:  1

In [13]:
# reftime: current time at time of prediction
# leadtime: number of hours after prediction time
# X, Y: (188, 127)
# chx, chy: in swiss coordinates
cosmo.isel(member=2, reftime=0, leadtime=1)

<xarray.Dataset>
Dimensions:   (x: 188, y: 127)
Coordinates:
    lat       (y, x) float32 dask.array<chunksize=(127, 188), meta=np.ndarray>
    leadtime  timedelta64[ns] 01:00:00
    lon       (y, x) float32 dask.array<chunksize=(127, 188), meta=np.ndarray>
    member    int32 2
    reftime   datetime64[ns] 2018-05-01
  * x         (x) int64 0 1 2 3 4 5 6 7 8 ... 180 181 182 183 184 185 186 187
  * y         (y) int64 0 1 2 3 4 5 6 7 8 ... 119 120 121 122 123 124 125 126
    chx       (y, x) float64 4.5e+05 4.522e+05 4.544e+05 ... 8.554e+05 8.577e+05
    chy       (y, x) float64 3.944e+04 3.951e+04 3.958e+04 ... 3.33e+05 3.33e+05
Data variables:
    PREC      (y, x) float32 dask.array<chunksize=(127, 188), meta=np.ndarray>

In [14]:
point = cosmo.isel(member=0, reftime=0, leadtime=1, x=0, y=0)
chx, chy = point['chx'], point['chy']
chx.values, chy.values

(array(449974.33803416), array(39435.62025801))

In [15]:
real_point = weather.isel(time=0, dummy=0).sel(chx=chx, chy=chy, method='nearest')
real_point['chx'].values, real_point['chy'].values

(array(449500.), array(39500.))

In [17]:
prec_pred = point['PREC'].values
prec_real = real_point['RR'].values
sklearn.metrics.brier_score_loss([prec_real], [prec_pred])

0.6722591355674581

In [18]:
### Scoring points

In [72]:
pred_points = cosmo.isel(reftime=0, leadtime=1)
chx, chy = pred_points['chx'], pred_points['chy']
# This is the grid along which our predictions are placed
chx.values, chy.values;
# These are the actual predictions (ensemble_cnt, x_len, y_len)
pred_points['PREC'].values.shape

(21, 127, 188)

In [75]:
real_points = weather.isel(time=0, dummy=0).sel(chx=chx, chy=chy, method='nearest')
# These are the values of the x and y coordinates 
# of the observations that are closest to our predictions.
real_points['chx'].values, real_points['chy'].values;
# These are the observations that are closest to our predictions.
# Note that the shape is the same as the shape of predictions
#  with the ensemble dimension removed.
real_points['RR'].values.shape

(127, 188)

In [79]:
import numpy as np

def mm_to_categorical(values):
    """Converts a tensor of precipitation values from mm to categorical."""
    # no rain (< 0.1mm / h), light rain (0.1 - 1 mm/h),  
    # rain( 1 - 5 mm/h), heavy rain (5 - 10 mm/h), 
    # very heavy rain (>= 10 mm/h)
    shape = values.shape# + (5,)
    y_cat = np.zeros(shape)
    y_cat[values < 0.1]                      = 0          # 0 - no rain
    y_cat[(0.1 <= values) & (values < 1.0)]  = 1  # 1 - light rain
    y_cat[(1.0 <= values) & (values < 5.0)]  = 2  # 2 - rain
    y_cat[(5.0 <= values) & (values < 10.0)] = 3  # 3 - heavy rain
    y_cat[10.0 <= values]                    = 4  # 4 - very heavy rain
    return y_cat

def categorical_to_probs(values):
    """Aggregates a tensor of categorical predictions for an ensemble to probability scores.
    The 'ensemble' elements should be placed in the first dimension. """
    y_probs = np.sum(values, axis=0) / values.shape[0]
    return y_probs


# These are the predicted classes from the ensemble
y_cat = mm_to_categorical(pred_points["PREC"].values)
# These are the class probabilities after averaging the ensemble
y_probs = categorical_to_probs(y_cat)

assert y_probs.shape == pred_points['PREC'].values.shape[1:]
print(y_probs.shape, pred_points['PREC'].values.shape)

(127, 188) (21, 127, 188)


In [81]:
# These are the true classes of the data
y_true = mm_to_categorical(real_points["RR"].values)

assert y_true.shape == y_probs.shape
y_true.shape

(127, 188)

In [104]:
# I don't understand this metric anyway
ps.threshold_brier_score(
    forecasts=pred_points['PREC'].values.swapaxes(0,-1), 
    observations=real_points['RR'].values.swapaxes(0,1), 
    threshold=0.1)

array([[0.14512472, 0.14512472, 0.14512472, ..., 0.        , 0.        ,
        0.        ],
       [0.05668934, 0.05668934, 0.05668934, ..., 0.        , 0.        ,
        0.        ],
       [0.00907029, 0.00907029, 0.03628118, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.02040816, 0.05668934,
        0.11111111],
       [0.        , 0.        , 0.        , ..., 0.02040816, 0.03628118,
        0.08163265],
       [0.        , 0.        , 0.        , ..., 0.02040816, 0.02040816,
        0.08163265]])